In [1]:
!apt update && apt install -y curl git

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,533 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,956 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/rest

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [7]:
!ollama pull llama3


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏  30 KB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏  11 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  55 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   3% ▕▏ 121 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   3% ▕▏ 137 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   4% ▕▏ 183 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 225 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 236 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 249 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 251 MB/4.7 GB  251 MB/s   

In [8]:
!pkill -f ngrok  # Kills all ngrok processes
!fuser -k 8501/tcp  # Kills any process using port 6000 (or use 5000 if needed)
# Install required libraries
OLLAMA_API_URL = "http://127.0.0.1:11434"
!nohup ollama serve > /dev/null 2>&1 &



In [5]:
!pip install streamlit PyPDF2 requests python-docx



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.8 MB/s eta 0:00:00


In [9]:
!pip install pyngrok

In [10]:
from pyngrok import ngrok

# Replace with your ngrok auth token
NGROK_AUTH_TOKEN = "2WG8G2YZNV9jofWBKu5B2E6hA3Q_719vVw9PLU1SwQLwCtcNd"

# Set authentication token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ ngrok Auth Token Set!")

public_url = ngrok.connect(8501).public_url
print(f"🚀 Streamlit App is running at: {public_url}")


✅ ngrok Auth Token Set!
🚀 Streamlit App is running at: https://2b3f-35-196-71-147.ngrok-free.app


In [11]:
import requests

OLLAMA_API_URL = "http://127.0.0.1:11434/api/generate"

headers = {"Content-Type": "application/json"}
payload = {
    "model": "llama3",
    "prompt": "What is the capital of France?",
    "stream": False
}

response = requests.post(OLLAMA_API_URL, json=payload, headers=headers)

if response.status_code == 200:
    print("✅ Response from Ollama:", response.json()["response"])
else:
    print(f"❌ Error from Ollama: {response.status_code}, {response.text}")


✅ Response from Ollama: The capital of France is Paris.


In [14]:
%%writefile app.py
import streamlit as st
import PyPDF2
import requests
import tempfile
import os

OLLAMA_URL = "http://127.0.0.1:11434/api/generate"
# Streamlit UI
st.set_page_config(page_title="Chat with PDF (Llama 3)", layout="wide")
st.title("📄 Chat with Your PDF (Llama 3)")
st.caption("Upload a PDF and ask questions about its content!")

uploaded_file = st.file_uploader("📂 Upload a PDF", type=["pdf"])

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() or ""  # Handle NoneType
    return text.strip()

if uploaded_file:
    st.success(f"✅ Uploaded: {uploaded_file.name}")

    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
        temp_file.write(uploaded_file.read())
        temp_pdf_path = temp_file.name

    pdf_text = extract_text_from_pdf(temp_pdf_path)
    os.remove(temp_pdf_path)

    if pdf_text:
        st.info("📄 PDF text extracted successfully!")
        with st.expander("🔍 View Extracted Text"):
            st.text_area("PDF Content", pdf_text[:1000] + "..." if len(pdf_text) > 1000 else pdf_text, height=200)

        question = st.text_input("💡 Ask a question about the document")

        if question:
            st.info("🤖 Sending question to Llama 3...")

            ollama_payload = {
                "model": "llama3",
                "prompt": f"Context: {pdf_text}\n\nQuestion: {question}\n\nAnswer:",
                "stream": False
            }
            response = requests.post(OLLAMA_URL, json=ollama_payload)

            if response.status_code == 200:
                result = response.json().get("response", "⚠️ No response from Llama 3.")
                st.subheader("📌 Answer:")
                st.write(result)
            else:
                st.error("⚠️ Failed to get response from Llama 3. Is Ollama running?")


Writing app.py


In [ ]:
!streamlit run app.py &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.196.71.147:8501

